<h4 style="font-variant-caps: small-caps;font-size:35pt;">MLlib Grid search</h4>

<span><i>Note that the content of this notebook is inspired from the notebook used in the Databricks course <b>Scalable Machine Learning with Apache Spark™ (V2)</b> available <a href="https://customer-academy.databricks.com/learn/course/1322/scalable-machine-learning-with-apache-spark-v2" target='_blank'>here</a></i>.</span>

<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">1. Import libraries</span></div>

In [ ]:
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator

<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">2. Load dataset</span></div>

In [ ]:
amsterdam_airbnb_df_url = "http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2023-09-03/visualisations/listings.csv"
amsterdam_airbnb_pandas_df = pd.read_csv(amsterdam_airbnb_df_url)

<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">3. Drop some columns</span></div>

In [ ]:
columns_to_exclude = ["id",
                      "name",
                      "host_id",
                      "host_name",
                      "neighbourhood_group",
                      "license",
                      "last_review",
                      "reviews_per_month"]
#
amsterdam_airbnb_pandas_df = amsterdam_airbnb_pandas_df.drop(columns=columns_to_exclude)

<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">4. Convert to Spark dataframe</span></div>

In [ ]:
schema = StructType([
    StructField("neighbourhood", StringType(), nullable=True),
    StructField("latitude", DoubleType(), nullable=True),
    StructField("longitude", DoubleType(), nullable=True),
    StructField("room_type", StringType(), nullable=True),
    StructField("price", IntegerType(), nullable=True),
    StructField("minimum_nights", IntegerType(), nullable=True),
    StructField("number_of_reviews", IntegerType(), nullable=True),
    StructField("calculated_host_listings_count", IntegerType(), nullable=True),
    StructField("availability_365", IntegerType(), nullable=True),
    StructField("number_of_reviews_ltm", IntegerType(), nullable=True)
])

In [ ]:
amsterdam_airbnb_df = spark.createDataFrame(amsterdam_airbnb_pandas_df, schema=schema)

<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">5. Optional: write to delta table</span></div>

In [ ]:
(amsterdam_airbnb_df.write
                    .mode("overwrite")
                    .option("overwriteSchema", "True")
                    .format("delta")
                    .saveAsTable("amsterdam_airbnb_df"))
#
amsterdam_airbnb_df = spark.table("amsterdam_airbnb_df")

<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">6. Prepare for ML</span></div>
<ul>
<li>The exercise here is to make a <b>binary classification</b>. A fake column named <code>priceClass</code> is created from <code>price</code> column. It doesn't make any particular sense. It can take two values - <code>true</code> or <code>false</code> - depending on <code>price</code> above or below <code>150</code>.</li>
<li>Note that converting the boolean column to type integer with <code>.cast("int")</code> automatically changes <code>true</code> to <code>1</code> and <code>false</code> to <code>0</code>.</li>
</ul>

In [ ]:
airbnb_df = (amsterdam_airbnb_df.withColumn("priceClass", (col("price") >= 150).cast("int"))
                                .drop("price"))

train_df, test_df = airbnb_df.randomSplit([.8, .2], seed=42)

categorical_cols = [field for (field, dataType) in train_df.dtypes if dataType == "string"]
index_output_cols = [x + "Index" for x in categorical_cols]

string_indexer = StringIndexer(inputCols=categorical_cols, outputCols=index_output_cols, handleInvalid="skip")

numeric_cols = [field for (field, dataType) in train_df.dtypes if ((dataType in ["double", "int"]) & (field != "priceClass"))]
assembler_inputs = index_output_cols + numeric_cols
vec_assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")

<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">7. Instantiate ML model: Random Forest classifier</span></div>
<ul>
<li>The following command let us verify that the minimum number of bins needed for hyperparameter <code>maxBins</code> of <code>RandomForestClassifier</code> model is <code>22</code>. It corresponds to the maximum number of unique values for columns of type string.</li>
</ul>

In [ ]:
count_distinct = [(column, amsterdam_airbnb_df.select(column).distinct().count(), amsterdam_airbnb_df.select(column).dtypes[0][-1]) for column in amsterdam_airbnb_df.columns]
display(spark.createDataFrame(count_distinct, ['Column', 'Distinct values', 'type']).orderBy(['type', 'Distinct values'], ascending=[0, 1]))

Column,Distinct values,type
room_type,4,string
neighbourhood,22,string
calculated_host_listings_count,17,int
minimum_nights,50,int
number_of_reviews_ltm,141,int
availability_365,366,int
number_of_reviews,485,int
price,631,int
latitude,5865,double
longitude,6845,double


<div>Then for the exercise, let's use <code>22</code> as value for <code>maxBins</code> hyperparameter.</div>

In [ ]:
rf = RandomForestClassifier(labelCol="priceClass", maxBins=22, seed = 42)

<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">8. Prepare for Grid Search</span></div>
<ul>
<li>Defining the grid as shown in the next cell will result in the training of 9 models: There are 3 x 3 parameter combinations.</li>
</ul>
<a id="gridsearch"></a>

In [ ]:
grid = (ParamGridBuilder().addGrid(rf.maxDepth, [2, 5, 10])
                          .addGrid(rf.numTrees, [10, 20, 100]).build())

<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">9. Prepare evaluator</span></div>
<ul>
<li>The metric chosen here is <code>area under ROC</code>.</li>
</ul>

In [ ]:
metric = "areaUnderROC"

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol="priceClass", metricName=metric)

<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">10. Definition of the Cross Validator</span></div>
<ul>
<li>Setting <code>numFolds</code> hyperparameter to 3 in the <code>CrossValidator</code> results in the training of 3 models, each of them trained on different set of rows in the dataset. It then results in a score which is the average of the three models scores.</li>
<li>When combined with a <b>Grid Search</b>, the number of models trained will be the number of <code>numFolds</code> hyperparameter of the <code>CrossValidator</code> multiplied by the number of <b>Grid Search</b> parameter combinations.</li>
<li>Then, in the particular case of this example, there will be 3 x 3 x 3 models to be trained, which is <b>27 models</b>.</li>
</ul>

In [ ]:
cv = CrossValidator(estimator=rf, evaluator=evaluator, estimatorParamMaps=grid, seed=42, numFolds=3)

<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">11. Definition of the pipeline and fit the pipeline</span></div>

In [ ]:
stages = [string_indexer, vec_assembler, cv]
#
pipeline = Pipeline(stages=stages)
#
pipeline_model = pipeline.fit(train_df)

<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">12. Get Grid Search parameters value of the best model</span></div>
<ul>
<li>There are 9 hyperparameters combinations in the <b>Grid Search</b>, thus 9 different model parameterizations.</li>
<li>Each of these 9 model parameterization is trained 3 times each time using a different set of data from the dataset, according to the value of <code>numFolds</code> hyperparameter of the <code>CrossValidator</code>.</li>
<li>The average of the 3 cross validations for each of the 9 models is the final result.</li>
<li>Consequently at the end, 27 models are trained resulting in 9 scores.</li>
</ul>

In [ ]:
columns_name = [paramName.name for paramName in list(pipeline_model.stages[-1].getEstimatorParamMaps())[0]] + [metric]

<span>Best model is the model with the highest value for <b>area under ROC</b>. It is obtained with the parameters from the <b>Grid Search</b> shown in the first row of the below table.</span>

In [ ]:
sets = [tuple([(v) for k,v in paramset.items()]+[str(avgmetric)]) for paramset,avgmetric in zip(list(pipeline_model.stages[-1].getEstimatorParamMaps()), pipeline_model.stages[-1].avgMetrics)]
#
display(spark.createDataFrame(sets, columns_name).orderBy(desc(metric)))

maxDepth,numTrees,areaUnderROC
10,100,0.8407864668331569
10,20,0.8327511187299091
10,10,0.8264245347627136
5,100,0.8207445767017671
5,20,0.8170598033231653
5,10,0.8132904263828932
2,100,0.7509357579175512
2,10,0.7364821016113127
2,20,0.7038806386896654


<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">13. More detailed information related to the best model</span></div>

In [ ]:
cv_model = pipeline_model.stages[-1]
rf_model = cv_model.bestModel
print(rf_model.explainParams())

bootstrap: Whether bootstrap samples are used when building trees. (default: True)
cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the featur

<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">14. Features by order of importance</span></div>

In [ ]:
pandas_df = pd.DataFrame(list(zip(vec_assembler.getInputCols(), rf_model.featureImportances)), columns=["feature", "importance"])
top_features = pandas_df.sort_values(["importance"], ascending=False)
top_features

,feature,importance
1,room_typeIndex,0.207192
0,neighbourhoodIndex,0.141898
5,number_of_reviews,0.135982
7,availability_365,0.127502
8,number_of_reviews_ltm,0.107315
3,longitude,0.091208
2,latitude,0.075542
4,minimum_nights,0.068298
6,calculated_host_listings_count,0.045064
